## Spark Dataframes
- Look like pandas dataframes
- share some of the same methods and syntax
- but they are 2 seperate types of objects

In [1]:
# Create Spark Session
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [2]:
# Imports

import pandas as pd
import numpy as np

## Exercises

1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [3]:
pd_df = pd.DataFrame ({"language": ["python", "java", "javascript", "scala", "R"]})

In [4]:
sp_df = spark.createDataFrame(pd_df)

In [5]:
sp_df

DataFrame[language: string]

In [6]:
# Use .show to view the dataframe
sp_df.show(5)

+----------+
|  language|
+----------+
|    python|
|      java|
|javascript|
|     scala|
|         R|
+----------+



In [7]:
sp_df.printSchema()

root
 |-- language: string (nullable = true)



In [8]:
print((sp_df.count(), len(sp_df.columns)))

(5, 1)


2. Load the mpg dataset as a spark dataframe.

    a. Create 1 column of output that contains a message like the one below:


The 1999 audi a4 has a 4 cylinder engine.
For each vehicle.

    b. Transform the trans column so that it only contains either manual or auto.

In [9]:
from pydataset import data

In [10]:
mpg = spark.createDataFrame(data("mpg"))

In [11]:
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [12]:
mpg.select(mpg.year.cast("string"), mpg.manufacturer, mpg.model, mpg.cyl.cast("string")).show()

+----+------------+------------------+---+
|year|manufacturer|             model|cyl|
+----+------------+------------------+---+
|1999|        audi|                a4|  4|
|1999|        audi|                a4|  4|
|2008|        audi|                a4|  4|
|2008|        audi|                a4|  4|
|1999|        audi|                a4|  6|
|1999|        audi|                a4|  6|
|2008|        audi|                a4|  6|
|1999|        audi|        a4 quattro|  4|
|1999|        audi|        a4 quattro|  4|
|2008|        audi|        a4 quattro|  4|
|2008|        audi|        a4 quattro|  4|
|1999|        audi|        a4 quattro|  6|
|1999|        audi|        a4 quattro|  6|
|2008|        audi|        a4 quattro|  6|
|2008|        audi|        a4 quattro|  6|
|1999|        audi|        a6 quattro|  6|
|2008|        audi|        a6 quattro|  6|
|2008|        audi|        a6 quattro|  8|
|2008|   chevrolet|c1500 suburban 2wd|  8|
|2008|   chevrolet|c1500 suburban 2wd|  8|
+----+-----

In [28]:
from pyspark.sql.functions import col, expr, concat, lit

In [37]:
mpg.select(concat(lit("The "), mpg.year,lit(" "), mpg.manufacturer, 
                 lit(" "),mpg.model, lit(" has a "), 
                 mpg.cyl, lit(" cylinder engine"))).show(5,False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
+-----------------------------------------------------------------------------+
only showing top 5 rows



In [53]:
from pyspark.sql.functions import regexp_extract, regexp_replace
mpg.select("trans", regexp_extract('trans', r'^(\w+)\(', 1).alias('trans'),).show(truncate=False)

+----------+------+
|trans     |trans |
+----------+------+
|auto(l5)  |auto  |
|manual(m5)|manual|
|manual(m6)|manual|
|auto(av)  |auto  |
|auto(l5)  |auto  |
|manual(m5)|manual|
|auto(av)  |auto  |
|manual(m5)|manual|
|auto(l5)  |auto  |
|manual(m6)|manual|
|auto(s6)  |auto  |
|auto(l5)  |auto  |
|manual(m5)|manual|
|auto(s6)  |auto  |
|manual(m6)|manual|
|auto(l5)  |auto  |
|auto(s6)  |auto  |
|auto(s6)  |auto  |
|auto(l4)  |auto  |
|auto(l4)  |auto  |
+----------+------+
only showing top 20 rows



3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [54]:
tips = spark.createDataFrame(data("tips"))

In [56]:
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [69]:
((tips.where(tips.smoker == "Yes").count()) / tips.count()) * 100

38.114754098360656

In [77]:
tips_percentage = (tips.tip / tips.total_bill) * 100

In [79]:
tips.createOrReplaceTempView("tips_view")

In [82]:
spark.sql(
"""
SELECT *,
(tip / total_bill) * 100 AS tip_percentag
FROM tips_view""").show()

+----------+----+------+------+---+------+----+------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentag|
+----------+----+------+------+---+------+----+------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|5.9446733372572105|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|16.054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|16.658733936220845|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 13.97804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|14.680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4| 18.62396204033215|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2| 22.80501710376283|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|11.607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|13.031914893617023|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|21.853856562922868|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 16.65043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|14.180374361883

In [96]:
tips_agg = tips.groupBy("sex", "smoker").mean("tip", "total_bill")

In [101]:
tips_agg.show()

+------+------+------------------+------------------+
|   sex|smoker|          avg(tip)|   avg(total_bill)|
+------+------+------------------+------------------+
|  Male|    No|3.1134020618556706| 19.79123711340206|
|  Male|   Yes| 3.051166666666666|22.284499999999998|
|Female|    No| 2.773518518518518| 18.10518518518519|
|Female|   Yes| 2.931515151515151|17.977878787878787|
+------+------+------------------+------------------+



In [110]:
tips_agg.select(tips.sex, tips.smoker, (tips_agg["avg(tip)"] / tips_agg["avg(total_bill)"]) * 100).show()

+------+------+------------------------------------+
|   sex|smoker|((avg(tip) / avg(total_bill)) * 100)|
+------+------+------------------------------------+
|  Male|    No|                  15.731215001953384|
|  Male|   Yes|                  13.691878510474394|
|Female|    No|                  15.318918255461906|
|Female|   Yes|                   16.30623493518971|
+------+------+------------------------------------+

